# Real time Sign to Text (Eng Sign Lang.)
![png](eng_sign\sign_lang_tutorial.png)  
z와 j는 유사한 수어가 존재하고 움직이는 방식으로 구분하는거 같음

### 사용 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob


### Data Load

In [2]:
train = pd.read_csv("./eng_sign/sign_mnist_train.csv")
test = pd.read_csv("./eng_sign/sign_mnist_test.csv")

train.shape, test.shape

((27455, 785), (7172, 785))

In [3]:
display(train.sample(3))
display(test.sample(3))

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
19682,5,158,160,164,166,169,173,172,173,174,...,42,44,52,54,47,37,37,39,43,41
23237,11,151,154,157,160,163,165,165,169,173,...,210,209,208,209,209,209,207,206,206,206
24255,22,55,70,79,25,34,62,70,75,82,...,173,178,180,182,184,186,188,189,190,190


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
3455,22,7,40,57,57,61,63,65,75,88,...,113,84,65,54,73,153,153,155,155,154
1973,5,150,151,150,151,153,154,154,154,155,...,211,200,203,200,200,200,198,197,195,194
1484,14,165,169,172,174,177,179,180,182,184,...,211,214,214,215,214,214,214,214,214,214


### Label

In [7]:
labels = train["label"].values
unique_val = np.array(labels)

In [9]:
np.unique(unique_val)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)